In [5]:
import pandas as pd
import numpy as np

In [6]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)

## Analyze data

In [9]:
import pandas as pd
data = pd.read_parquet("data/prepared_data/prepared_df.parquet")

In [10]:
data.sample(10)

,place_id,review,question,tags,proba
26697,27,"Находится в центре города, недалеко от автобусной остановки. Эрвье был начальником Главного Тюме...",Что?,[памятник подходит объект фотографирования],"[0.469, 0.265]"
34849,91,"Запущенный сквер, но если скверик пролегает на пути вашего следования, то в сухую, теплую погоду...",Для чего?,None,None
10401,2,"Великолепное место. Визитная карточка города, присутствующая на большом числе рекламных проспект...",Какой?,[визитная карточка города],"[0.408, 0.23]"
31749,46,"раньше это был просто небольшой сквер с памятником Борцам революции, теперь же это еще и крутая ...",Для чего?,[крутая детская площадка],"[0.347, 0.383]"
8212,2,"Мост ведет на другую сторону реки, в район частного сектора и совершенно другой атмосферы, Много...",Когда?,[летом весной],"[0.915, 0.554]"
32369,51,Гулял вечером в близлежащем районе от моей гостиницы и случайно обнаружил этот оригинальный памя...,Для чего?,[напротив медицинской академии],"[0.305, 0.335]"
26475,27,"Памятник Юрию Эрвье колоритен. Он отражает тюменский дух. Некоторая суровость, мужественность лю...",Для кого?,[полноты впечатлений тюмени],"[0.779, 0.526]"
4226,1,Правильнее сказать самая дорогая. Она смотрится и правду очень дорого. Она шире чем сама река и ...,С кем?,[родители детьми],"[0.885, 0.773]"
35791,100,"Итак, #отзывыотнас . Кафе-боулинг Brooklyn Bowl. Сохраняем, чтоб не потерять)) и лайк 😍\n👌понрав...",Какой?,[кафе - боулинг brooklyn bowl],"[0.301, 0.228]"
2481,1,"В хорошую погоду здесь можно встретить почти всех жителей города), настолько популярно это место...",Какой?,[красивущую набережную построили очень удобную - ярусов],"[0.677, 0.221]"


---

In [299]:
import os
import re
os.chdir("../")

In [49]:
from src.models import roberta
from nltk.corpus import stopwords

In [82]:
model = roberta.Pipeline()

In [995]:
def return_tags(part: dict, root_dep: str) -> str:
        tags = []
        # для того, чтобы оставить только главное существительное
        root_stop = False
        # для того, чтобы возврщать слова из предложения, а не леммы
        return_keys = False
        single_word = True if len(part.keys()) == 1 else False
        for num, key in enumerate(part.keys()):

            if single_word:
                tags.append(part[key]["lemma"])
                continue

            if root_stop:
                break

            # should return keys?
            if (
                num == 0 and (
                    (part[key]["dep"] == "case" or part[key]["dep"] == "advmod")
                    or (part[key]["dep"] == "nsubj")
                    or (part[key]["pos"] == 'CCONJ' and part[key]["dep"] == 'cc')
                    or (part[key]["dep"] == 'ROOT')
                    or (part[key]["pos"] == 'ADJ', part[key]["dep"] == 'amod')
                )
            ):
                return_keys = True
    
            # should pass the word?
            if (
                (
                    part[key]["pos"] == "ADV" 
                    and (part[key]["dep"] == "advmod" or part[key]["dep"] == "punct")
                    and num+1 != len(part.keys()))
                or (part[key]["dep"] == ['nmod'] and num == 0)
                or (part[key]["pos"] == 'CCONJ' and part[key]["dep"] == 'cc' and num == 0)
                or (part[key]["dep"] == "mark" and root_dep == 'VERB')
            ):
                continue

            if root_dep == 'VERB':
                if return_keys:
                    tags.append(key)
                else:
                        tags.append(part[key]["lemma"])

                if part[key]["dep"] == 'xcomp' and len(tags) == 1:
                    tags.pop(0)
                    tags.append(key)
            
            elif root_dep == "NOUN":
                if return_keys:
                    tags.append(key)
                    if (
                        num != 0 
                        and part[key]["dep"] == "ROOT"
                        and num != len(part.keys()) - 2
                    ):
                        root_stop = True
                else:
                    tags.append(part[key]["lemma"])

            elif  root_dep == "ADJ":
                if part[key]["dep"] == "mark":
                    continue
                else:
                    tags.append(key) 

            elif  root_dep == "ADV":
                tags.append(key)

            elif  root_dep == "NUM":
                tags.append(key)  
                              
            else:
                tags.append(part[key]["lemma"])

        return " ".join(tags)

In [996]:
questions = [
    "что не понравилось в этом месте",
    "где находится это место",
    "что это за место",
    "чем можно заняться в этом месте",
    "в какое время лучше посетить это место",
    "как много денег стоит посетить это место",
    "с кем прийти в это место"]

In [997]:
example

'Очень простая по своей идее скульптура. И одновременно очень трогательная.\nСтоял изучал фотографии - и ком в горле'

In [1002]:
for i in data.sample(1).index:
    example = data.review[i]
    print(example, "\n")
    for question in questions:
        answer, probas = model.answer(example, question, 0.)



        if (
            not answer or
            answer == '' or
            answer == question.lower().replace('?', '')
        ):
            continue

        try:
            answer = answer.split(">")[1]
        except IndexError:
            pass
        # print(answer)

        stops = stopwords.words("russian")
        stops = [word for word in stops if word not in model._norm_words+["не", "на", "от", "а", "в", "за", "до", "с"]]

        corpus = answer.split(" ")
        text = " ".join([word for word in corpus if word not in stops])
        text_list = re.split(r', |\. |\.|,| а | и | просто | или |!|!!!| ! ', text)

        answer_cleared = text_list
        # print(text_list)

        parts = []
        roots_dep = []
        for answer in answer_cleared:
            document = model._nlp_model(answer)
            doc_dict = {}
            for token in document:
                if token.pos_ not in "PUNCT":
                    doc_dict[token.text] = {
                        "lemma": token.lemma_,
                        "pos": token.pos_,
                        "dep": token.dep_,
                    }
                    if token.dep_ == "ROOT":
                        roots_dep.append(token.pos_)
            parts.append(doc_dict)
        
        print(parts)

        tags = []
        for part, root_dep in zip(parts, roots_dep):
            if part:
                tags.append(return_tags(part, root_dep))
        
        print(question, tags, "\n", sep = "\n")

Трогательный монумент расположен в районе дома печати, на пересечении улиц Свердлова и Комсомольской. Очень приятная композиция 

[{'очень': {'lemma': 'очень', 'pos': 'ADV', 'dep': 'advmod'}, 'приятная': {'lemma': 'приятный', 'pos': 'ADJ', 'dep': 'amod'}, 'композиция': {'lemma': 'композиция', 'pos': 'NOUN', 'dep': 'ROOT'}}]
что не понравилось в этом месте
['приятная композиция']


[{'в': {'lemma': 'в', 'pos': 'ADP', 'dep': 'case'}, 'районе': {'lemma': 'район', 'pos': 'NOUN', 'dep': 'ROOT'}, 'дома': {'lemma': 'дом', 'pos': 'NOUN', 'dep': 'nmod'}, 'печати': {'lemma': 'печать', 'pos': 'NOUN', 'dep': 'nmod'}}]
где находится это место
['в районе']


[{'в': {'lemma': 'в', 'pos': 'ADP', 'dep': 'case'}, 'районе': {'lemma': 'район', 'pos': 'NOUN', 'dep': 'ROOT'}, 'дома': {'lemma': 'дом', 'pos': 'NOUN', 'dep': 'nmod'}, 'печати': {'lemma': 'печать', 'pos': 'NOUN', 'dep': 'nmod'}}]
что это за место
['в районе']


[{'в': {'lemma': 'в', 'pos': 'ADP', 'dep': 'case'}, 'районе': {'lemma': 'район', 'pos

In [740]:
answer

[]

In [723]:
answer

['свой фирменный стиль']